# Lab 3.4 - Azure AI Search with images

Azure AI Search can also store embeddings of images. This is useful for searching for images based on a text prompt or even another image. In this lab, we will use Azure AI Search to index images, store their embeddings and do semantic queries on them.

## Step 1 - Upload images

1. Go to the Azure Portal and open your storage account
2. Navigate to Storage Browser -> Blob Containers and click Add Container
3. Call it `images` and click OK
4. Open the container and upload the images in the `data/images` folder of this repo

![Storage](../img/si01.png)

## Step 2 - Create an Azure AI Services account

Because we'll be using a feature of Azure AI Vision to get image embeddings, we need to create an Azure AI Services account. This is a multi-service account that shares the same key for its services.

1. In the Azure Portal, click on Create a resource
2. Search for `Azure AI Services` and click Create
3. Choose the resource group and region (use the same as the AI Search service)
4. Choose Standard S0 as the pricing tier
5. For demo purposes, skip the rest of the wizard and click Review + Create
6. Click Create

![Storage](../img/si05.png)

## Step 3 - Index images in Azure AI Search

1. Go to the Azure Portal and open your Azure AI Search service
2. Click on Import and Vectorize Data
3. Choose the storage account and the `images` container, leave everything else blank on this page
4. For text vectorization, choose `Kind: OpenAI`, select your Azure OpenAI Service and an embeddings model

![Storage](../img/si02.png)

5. Click the `Vectorize images` checkbox
6. Choose `Kind: AI Vision vectorization`
7. Select your Azure Cognitive Services multi-service account

![Storage](../img/si03.png)

8. Leave Semantic Ranker enabled
9. Click Next 
10. Use `images` as the object name prefix (this will be the index name) and click Create

## Step 4 - See indexed data

1. Wait for a few seconds for data to be indexed 
2. Navigate to Indexes and choose the `images` index
3. Enter `*` in the search box and click Search
4. Notice how we now have an image_vector that contains the embeddings of the images

![Storage](../img/si04.png)

## Step 5 - Store Azure Storage account name and key

In your `.env` file add the info for the 2 keys:

* `AZURE_STORAGE_ACCOUNT_NAME` - the name of your storage account
* `AZURE_STORAGE_ACCOUNT_KEY` - the access key of your storage account

**NOTE:** After adding these entries, click on the `Restart` button in this jupyter notebook to reload the environment variables.

![Storage](../img/si06.png)

## Step 5 - Let's do some queries in code

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery, VectorizableTextQuery, VectorizableImageBinaryQuery
from azure.storage.blob import BlobServiceClient
from IPython.display import display, Image

# Load the environment variables with dotenv
from dotenv import load_dotenv
load_dotenv()
import os

# Azure AI Search info
service_endpoint = os.getenv("AZURE_AI_SEARCH_ENDPOINT")
key = os.getenv("AZURE_AI_SEARCH_API_KEY")
index_name = "images" # replace this if you named your index differently

# Create a client
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))

print('Using endpoint', service_endpoint)
print('Using index', index_name)

# Connect to storage account where images are stored
AZURE_STORAGE_ACCOUNT_NAME = os.getenv("AZURE_STORAGE_ACCOUNT_NAME")
AZURE_STORAGE_ACCOUNT_KEY = os.getenv("AZURE_STORAGE_ACCOUNT_KEY")

# Create a blob service client
blob_service_client = BlobServiceClient(account_url=f"https://{AZURE_STORAGE_ACCOUNT_NAME}.blob.core.windows.net", credential=AZURE_STORAGE_ACCOUNT_KEY)

print('Using storage account', AZURE_STORAGE_ACCOUNT_NAME)

In [ ]:
# A helper function to show images
def show_image(blob_name):
    blob_client = blob_service_client.get_blob_client("images", blob_name)
    stream = blob_client.download_blob().readall()
    display(Image(stream, width=200))

In [ ]:
# Define a helper function to search
# Notice:
#   - We are using the image_vector field to search
#   - We are using the selecting up to 5 results in k_nearest_neighbors
def search(query):
    r = search_client.search(
        vector_queries=[VectorizableTextQuery(text=query, k_nearest_neighbors=5, fields="image_vector")]
    )
    for doc in r:
        # Because this is a vector search, there will always be results so we define a mininum score to only show relevant results
        score_threshold = 0.6
        if doc["@search.score"] < score_threshold:
            break
        print(doc["@search.score"],doc["title"], doc["metadata_storage_path"])
        show_image(doc["title"])

In [ ]:
search("do you have any backpacks?")

In [ ]:
# By asking specifically for red, we should get zero results
search("do you have any red backpacks?")

In [ ]:
search("I want to go to the mountains. What can you show me?")

In [ ]:
search("do you have something for animal lovers?")

In [ ]:
search("I'd love to go fishing!")

In [ ]:
search("What abot music?")